In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier


In [2]:
col_names = ['Subject',
             'Experiment ID',
             'Feature extraction',
             'Band selection',
             'Channel selection',
             'Classification algorithm',
             'Accuracy']

1 subject.
No feature extraction.

In [4]:
dictRaw = pd.read_pickle('d\\s02.dat') 
labels = dictRaw.get('labels')
data = dictRaw.get('data')
        
dfLabels = pd.DataFrame(data=labels, columns=["valence", "arousal", "dominance", "liking"])

a,b,c = data.shape
E = data.reshape(40, 322560)
allData = pd.DataFrame(E).transpose()
rows = list()
for _,row in dfLabels.iterrows():
    rows += [row]*8064
aux = pd.DataFrame(rows).reset_index(drop=True)
data = allData.merge(aux, left_index=True, right_index=True)

data.columns = ['FP1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3','Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8','FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz','hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature','valence', 'arousal', 'dominance', 'liking']

data.drop(['hEOG','vEOG','zEMG','tEMG','GSR','Respiration','PLethy','Temperature', 'arousal', 'dominance', 'liking'],axis = 'columns', inplace=True)
#data.drop(['arousal', 'dominance', 'liking'], axis=1)

data['valence'] = np.where(data['valence'] <5, 'sad', 'happy')

copia = data.copy()

y = copia[['valence']]

x = copia.drop(columns=['valence'])

xTrain, xTest, yTrain, yTest = train_test_split(x,y, test_size = 0.2, random_state = 1)
        

# Random Forest

In [6]:
def clas_RF(xTrain, yTrain, xTest, yTest):
    clf = RandomForestClassifier(max_depth=2, random_state=0, verbose=1, n_jobs=5)
    clf.fit(xTrain, yTrain)
    predict = clf.predict(xTest)


    rf_acc = accuracy_score(yTest, predict)
    return rf_acc

<ipython-input-6-a0276de5f258>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(xTrain, yTrain)
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    5.2s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:   13.1s finished
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    0.1s finished


0.7393973214285714

# SVM

In [7]:
def clas_SVM(xTrain, yTrain, xTest, yTest):
    svm_clf = svm.LinearSVC()
    svm_clf.fit(xTrain, np.ravel(yTrain))
    svm_predict = svm_clf.predict(xTest)
    svm_acc = accuracy_score(yTest, svm_predict)
    return svm_acc

# kNN

In [27]:
def clas_kNN(xTrain, yTrain, xTest, yTest):

     n_neighbors = 7

     knn = KNeighborsClassifier(n_neighbors)
     knn.fit(xTrain, np.ravel(yTrain))
     knn_predict = knn.predict(xTest)
     knn_acc = accuracy_score(yTest, knn_predict)
     return knn_acc

0.7883029513888888

In [28]:
def write_csv():
    data_CSV = [subject_id, 1, "-", "-", "-", "Random Forest", rf_acc], [subject_id, 1, "-", "-", "-", "SVM", svm_acc], [subject_id, 1, "-", "-", "-", "kNN", knn_acc]
    all_data += data_CSV

    csv_df = pd.DataFrame(all_data, columns=col_names)

    csv_df.to_csv("results.csv", index=False)

(['2', 1, '-', '-', '-', 'Random Forest', 0.7393973214285714],
 [1, 1, '-', '-', '-', 'SVM', 0.5614769345238095],
 [1, 1, '-', '-', '-', 'kNN', 0.7883029513888888])